In [2]:
# Importation des bibliothèques nécessaires
from bs4 import BeautifulSoup
import pandas as pd
from operator import concat
from datetime import datetime, timedelta

# Chargement du contenu HTML depuis un fichier local
try:
    with open("page_content.html", "r", encoding="utf-8") as file:
        html_content = file.read()
except FileNotFoundError:
    raise FileNotFoundError("Le fichier 'page_content.html' est introuvable. Vérifiez le chemin et réessayez.")
except Exception as e:
    raise Exception(f"Erreur lors de la lecture du fichier HTML : {e}")

# Analyse du contenu HTML avec BeautifulSoup
try:
    soup = BeautifulSoup(html_content, 'html.parser')
except Exception as e:
    raise Exception(f"Erreur lors de l'analyse du contenu HTML : {e}")

# Sélection de l'élément contenant les données
tabs = soup.select_one("div.tabs")
if not tabs:
    raise ValueError("Aucun élément 'div.tabs' trouvé dans le fichier HTML.")

# Récupération des éléments visibles et cachés
try:
    valuebets = concat(tabs.select("div.visible"), tabs.select("div.hidden"))
except Exception as e:
    raise Exception(f"Erreur lors de la récupération des éléments visibles et cachés : {e}")

# Initialisation du dictionnaire pour stocker les données
data = {
    'sports': [],
    'countries': [],
    'leagues': [],
    'pronos': [],
    'date': [],
    'time': [],
    'team_1': [],
    'team_2': [],
    'outcome': [],
    'bookmaker': [],
    'odds': [],
    'value': [],
    'probability': []
}

# Extraction des données des paris
for valuebet in valuebets:
    try:
        header = valuebet.select("a")
        data['sports'].append(header[0].text.strip() if len(header) > 0 else None)
        data['countries'].append(header[1].text.strip() if len(header) > 1 else None)
        data['leagues'].append(' '.join(header[2].text.split()) if len(header) > 2 else None)
    except Exception as e:
        print(f"Erreur lors de l'extraction des données de l'en-tête : {e}")

# Extraction des informations sur les matchs
match_info = tabs.find_all("div", class_="flex min-h-[90px] w-full")
for match in match_info:
    try:
        p_elements = match.select("p")
        match_data = [p.text.strip() for p in p_elements]
        if len(match_data) >= 9:  # Vérification pour éviter les erreurs d'indice
            data['pronos'].append(match_data[0])
            data['date'].append(match_data[1])
            data['time'].append(match_data[2])
            data['team_1'].append(match_data[3])
            data['team_2'].append(match_data[4])
            data['outcome'].append(match_data[5])
            data['odds'].append(match_data[6])
            data['value'].append(match_data[7])
            data['probability'].append(match_data[8])
    except Exception as e:
        print(f"Erreur lors de l'extraction des informations sur les matchs : {e}")

# Extraction des informations sur les bookmakers
bookmaker_info = tabs.find_all("div", class_="h-[25px] w-[75px]")
for bookmaker in bookmaker_info:
    try:
        img = bookmaker.find("img")
        data['bookmaker'].append(img['alt'] if img and 'alt' in img.attrs else None)
    except Exception as e:
        print(f"Erreur lors de l'extraction des informations sur les bookmakers : {e}")

# Création d'un DataFrame pandas à partir des données collectées
try:
    full_df = pd.DataFrame(data)
except Exception as e:
    raise Exception(f"Erreur lors de la création du DataFrame : {e}")

# Nettoyage des données
df = full_df.copy()

# Nettoyage de la colonne "probability" pour enlever le caractère '%'
df['probability'] = pd.to_numeric(df['probability'].apply(lambda p: p.replace('%', '') if isinstance(p, str) else None), errors='coerce')

# Remplacement de "Tomorr." par la date de demain dans la colonne "date"
today = datetime.now()
df['date'] = df['date'].apply(lambda date: date.replace(',', '').replace("Tomorr.", (today + timedelta(days=1)).strftime('%d %b')) if isinstance(date, str) else None)

# Ajout de l'année actuelle aux dates
current_year = today.year
df['date'] = df['date'].apply(lambda date: f"{date} {current_year}" if isinstance(date, str) else None)

# Uniformisation du format des colonnes "date" et "time"
df['date'] = pd.to_datetime(df['date'], format='%d %b %Y', errors='coerce')
df['time'] = pd.to_datetime(df['time'], format='%H:%M', errors='coerce').dt.strftime('%H:%M')

# Conversion des colonnes "value" et "odds" en float
df['value'] = pd.to_numeric(df['value'], errors='coerce')
df['odds'] = pd.to_numeric(df['odds'], errors='coerce')

# Sauvegarde du DataFrame dans un fichier CSV
try:
    df.to_csv('oddsportal_data.csv', index=False)
    print("Les données ont été sauvegardées avec succès dans 'oddsportal_data.csv'.")
except Exception as e:
    raise Exception(f"Erreur lors de la sauvegarde des données dans un fichier CSV : {e}")

# Affichage des 5 premières lignes du DataFrame
df.head()

Les données ont été sauvegardées avec succès dans 'oddsportal_data.csv'.


,sports,countries,leagues,pronos,date,time,team_1,team_2,outcome,bookmaker,odds,value,probability
0,Football,Thailand,Thai League 1,"Home/Away, FT including OT",2025-04-26,02:00,Montreal Canadiens,Washington Capitals,1,BC.Game,6.5,3.13,46.49
1,Football,Brazil,Serie C,"O/U 4.5, 1st Half",2025-04-27,14:00,Muang Thong Utd,Nakhon Ratchasima FC,Over,1xBet,41.0,2.60,7.65
2,Football,Africa,CAF Champions League,"O/U 3.5, 1st Half",2025-04-27,01:30,CSA,Nautico,Over,1xBet,34.0,2.45,7.91
3,Football,Brazil,Serie C,"O/U 3.5, 1st Half",2025-04-26,21:00,Pyramids,Orlando Pirates,Over,1xBet,34.0,2.42,7.80
4,Football,Thailand,Thai League 1,"O/U 3.5, 1st Half",2025-04-27,22:30,Itabaiana,Sao Bernardo,Over,1xBet,34.0,2.42,7.78
